In [1]:
import pandas as pd
import numpy as np

In [2]:
original_df = pd.read_csv('output_0831.csv', sep='\t')
df_gt = pd.read_csv('data-ground-truth-11-28.tsv', sep='\t')
# preprocess
original_df = original_df.drop_duplicates(subset=['id_annotator', 'caption', 'url'], keep='last')
cols = {'cb0':'Meta', 'cb1':'True', 'cb2':'Action', 
        'cb3':'Subjective', 'cb4':'Story', 'cb5':'Irrelevant', 
        'cb6':'Other', 'cbWhen':'When', 'cbHow':'How', 
        'cbWhere':'Where', 'other':'Comment'}
original_df.rename(columns=cols, inplace=True)
original_df.Comment = original_df.Comment.str.strip().str.lower()
new_cols = ['id_annotation', 'id_annotator', 'caption', 'url', 'True', 'Subjective', 'Action', 'Story', 'Meta', 'Irrelevant', 'Other', 'Comment', 'When', 'How', 'Where', 'Identification', 'source', 'cbBrokenImg']
df_all = original_df[new_cols]
df_gt = df_gt[new_cols[:-1]]
categories = list(df_all.columns)
print(df_all.shape, categories)

(13858, 18) ['id_annotation', 'id_annotator', 'caption', 'url', 'True', 'Subjective', 'Action', 'Story', 'Meta', 'Irrelevant', 'Other', 'Comment', 'When', 'How', 'Where', 'Identification', 'source', 'cbBrokenImg']


In [3]:
df_pn = df_all[df_all.id_annotation >= 11651]
df_pn = df_pn[df_pn.id_annotator.isin(['Kat', 'Ilana'])]
df_pn = df_pn[df_pn.cbBrokenImg == 0]

In [4]:
import spacy

nlp = spacy.load("en_core_web_sm")

In [5]:
def get_pronoun_dist(df_t, cols):
    PRONOUN_TAGS = ['DET', 'PRON']
    PRONOUN_TAGS2 = ['PRP', 'PRP$', 'WP', 'WP$']
    counter = {}
    counter2 = {}
    columns = df_t.columns[cols].to_list()
    for col in columns:
        counter[col] = 0
        counter2[col] = 0
    for index, row in df_t.iterrows():
        text = row.caption
        doc = nlp(text)
        flag = False
        flag2 = False
        for token in doc:
            if token.pos_ in PRONOUN_TAGS:
                flag = True
            if token.tag_ in PRONOUN_TAGS2:
                flag2 = True
        if flag:
            for col in columns:
                if row[col] == 1:
                    counter[col] += 1
        if flag2:
            for col in columns:
                if row[col] == 1:
                    counter2[col] += 1
    print('Stats that includes determiners (get by tags [\'PRON\', \'DET\'] from spaCy')
    for col in columns:
        print('{:12} {:8.2f}%'.format(col + ':', 100.0 * counter[col] / len(df_gt)))
    print('\nStats that doesn\'t include determiners (get by pos tags [\'PRP\', \'PRP$\', \'WP\', \'WP$\'] from spaCy)')
    for col in columns:
        print('{:12} {:8.2f}%'.format(col + ':', 100.0 * counter2[col] / len(df_gt)))

In [ ]:
cols = list(np.arange(4, 11)) + list(np.arange(12, 16))
get_pronoun_dist(df_gt, cols)

In [ ]:
get_pronoun_dist(df_pn, cols)